In [1]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score,recall_score

/Users/sgcx198/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Users/sgcx198/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
path = '../data/'
train = pd.read_csv(path+'train_set.csv')
test = pd.read_csv(path+'test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
ID           25317 non-null int64
age          25317 non-null int64
job          25317 non-null object
marital      25317 non-null object
education    25317 non-null object
default      25317 non-null object
balance      25317 non-null int64
housing      25317 non-null object
loan         25317 non-null object
contact      25317 non-null object
day          25317 non-null int64
month        25317 non-null object
duration     25317 non-null int64
campaign     25317 non-null int64
pdays        25317 non-null int64
previous     25317 non-null int64
poutcome     25317 non-null object
y            25317 non-null int64
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 17 columns):
ID           10852 non-null int64
age          10852 non-null int64
job          10852 non-null object
mar

In [3]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [4]:
test.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [5]:
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [6]:
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [8]:
test['y'] = -1
print(len(test.columns))

18


In [9]:
data = train.append(test).reset_index(drop=True)
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [10]:
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder

In [11]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl  = LabelEncoder()
    data['count_'+i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

In [12]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


In [13]:
feats = [i for i in data.columns if i not in ['ID','y']]
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'count_job',
 'count_marital',
 'count_education',
 'count_default',
 'count_housing',
 'count_loan',
 'count_contact',
 'count_month',
 'count_poutcome']

In [14]:
model = lgb.LGBMClassifier(
    boosting_type='gbdt',num_leaves = 30,reg_alpha=0.,
    max_depth = -1,n_estimators = 1500,objective='binary',metric='auc',
    subsample=0.95,colsample_bytree=0.7,subsample_freq=1,
    learning_rate=0.02,random_state=2017
)
train_x = data[data['y'] != -1][feats]
train_y = data[data['y'] != -1]['y']
test_x = data[data['y'] == -1][feats]
model.fit(train_x,train_y)
test_pre = model.predict_proba(test_x)[:,1]

In [15]:
print(len(test_pre))

10852


In [17]:
pre = data[data['y'] == -1][['ID']]
pre['pred'] = test_pre
print(len(pre))

10852


In [18]:
pre.head()

,ID,pred
25317,25318,0.053728
25318,25319,0.003968
25319,25320,0.001854
25320,25321,0.785342
25321,25322,0.013772


In [19]:
# pre.to_csv('baseline.csv',index=False)

In [20]:
# 五折交叉验证
from sklearn.model_selection import KFold
n_splits = 10
kfold = KFold(n_splits=n_splits,shuffle=True,random_state=42)
train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']
res=data[data['y']==-1][['ID']]

test_x= data[data['y']==-1][feats]
res['pred'] = 0
for train_idx,val_idx in kfold.split(train_x):
    model.random_stete = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    model.fit(train_x1,train_y1,eval_set=[(train_x1,train_y1),(test_x1,test_y1)],
             eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]
res['pred'] = res['pred'] / 10
# res.to_csv('sub_fold.csv',index=False)

[1]	training's auc: 0.889446	valid_1's auc: 0.88118
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.909064	valid_1's auc: 0.891137
[3]	training's auc: 0.914388	valid_1's auc: 0.901286
[4]	training's auc: 0.915725	valid_1's auc: 0.904565
[5]	training's auc: 0.925645	valid_1's auc: 0.915956
[6]	training's auc: 0.925366	valid_1's auc: 0.915838
[7]	training's auc: 0.927597	valid_1's auc: 0.915322
[8]	training's auc: 0.927806	valid_1's auc: 0.915095
[9]	training's auc: 0.92745	valid_1's auc: 0.91312
[10]	training's auc: 0.928673	valid_1's auc: 0.91437
[11]	training's auc: 0.929093	valid_1's auc: 0.91465
[12]	training's auc: 0.929535	valid_1's auc: 0.915321
[13]	training's auc: 0.929723	valid_1's auc: 0.915525
[14]	training's auc: 0.930574	valid_1's auc: 0.916872
[15]	training's auc: 0.931027	valid_1's auc: 0.917423
[16]	training's auc: 0.931157	valid_1's auc: 0.917665
[17]	training's auc: 0.93131	valid_1's auc: 0.918127
[18]	training's auc: 0.932162	valid

[186]	training's auc: 0.953204	valid_1's auc: 0.932384
[187]	training's auc: 0.953304	valid_1's auc: 0.93233
[188]	training's auc: 0.953372	valid_1's auc: 0.932305
[189]	training's auc: 0.953461	valid_1's auc: 0.932268
[190]	training's auc: 0.953526	valid_1's auc: 0.932252
[191]	training's auc: 0.953586	valid_1's auc: 0.932362
[192]	training's auc: 0.953649	valid_1's auc: 0.932337
[193]	training's auc: 0.953716	valid_1's auc: 0.932358
[194]	training's auc: 0.953778	valid_1's auc: 0.932319
[195]	training's auc: 0.953839	valid_1's auc: 0.932343
[196]	training's auc: 0.953956	valid_1's auc: 0.932459
[197]	training's auc: 0.954034	valid_1's auc: 0.932484
[198]	training's auc: 0.954104	valid_1's auc: 0.932505
[199]	training's auc: 0.954187	valid_1's auc: 0.932492
[200]	training's auc: 0.954233	valid_1's auc: 0.932476
[201]	training's auc: 0.954337	valid_1's auc: 0.932586
[202]	training's auc: 0.954399	valid_1's auc: 0.932596
[203]	training's auc: 0.95447	valid_1's auc: 0.932568
[204]	traini

[339]	training's auc: 0.963508	valid_1's auc: 0.933107
[340]	training's auc: 0.963587	valid_1's auc: 0.933203
[341]	training's auc: 0.963638	valid_1's auc: 0.933148
[342]	training's auc: 0.963686	valid_1's auc: 0.933122
[343]	training's auc: 0.963724	valid_1's auc: 0.933144
[344]	training's auc: 0.963814	valid_1's auc: 0.933229
[345]	training's auc: 0.963885	valid_1's auc: 0.933201
[346]	training's auc: 0.963961	valid_1's auc: 0.933215
[347]	training's auc: 0.964051	valid_1's auc: 0.933178
[348]	training's auc: 0.964117	valid_1's auc: 0.933222
[349]	training's auc: 0.964181	valid_1's auc: 0.933248
[350]	training's auc: 0.964244	valid_1's auc: 0.933243
[351]	training's auc: 0.964317	valid_1's auc: 0.933248
[352]	training's auc: 0.964379	valid_1's auc: 0.933243
[353]	training's auc: 0.964443	valid_1's auc: 0.933175
[354]	training's auc: 0.964498	valid_1's auc: 0.933145
[355]	training's auc: 0.964548	valid_1's auc: 0.93317
[356]	training's auc: 0.964631	valid_1's auc: 0.93321
[357]	traini

[503]	training's auc: 0.972083	valid_1's auc: 0.93387
[504]	training's auc: 0.972121	valid_1's auc: 0.933902
[505]	training's auc: 0.972173	valid_1's auc: 0.933879
[506]	training's auc: 0.972215	valid_1's auc: 0.933898
[507]	training's auc: 0.972239	valid_1's auc: 0.93389
[508]	training's auc: 0.972282	valid_1's auc: 0.933937
[509]	training's auc: 0.972333	valid_1's auc: 0.933996
[510]	training's auc: 0.972387	valid_1's auc: 0.934043
[511]	training's auc: 0.972439	valid_1's auc: 0.934077
[512]	training's auc: 0.972471	valid_1's auc: 0.934073
[513]	training's auc: 0.972522	valid_1's auc: 0.934063
[514]	training's auc: 0.972566	valid_1's auc: 0.93403
[515]	training's auc: 0.972623	valid_1's auc: 0.934029
[516]	training's auc: 0.972653	valid_1's auc: 0.934074
[517]	training's auc: 0.972696	valid_1's auc: 0.934126
[518]	training's auc: 0.972737	valid_1's auc: 0.934158
[519]	training's auc: 0.972787	valid_1's auc: 0.934126
[520]	training's auc: 0.972829	valid_1's auc: 0.934079
[521]	trainin

[666]	training's auc: 0.978555	valid_1's auc: 0.934326
[667]	training's auc: 0.978592	valid_1's auc: 0.934363
[668]	training's auc: 0.978624	valid_1's auc: 0.934376
[669]	training's auc: 0.978643	valid_1's auc: 0.934353
[670]	training's auc: 0.978675	valid_1's auc: 0.934404
[671]	training's auc: 0.978726	valid_1's auc: 0.934391
[672]	training's auc: 0.978759	valid_1's auc: 0.93437
[673]	training's auc: 0.978784	valid_1's auc: 0.934401
[674]	training's auc: 0.978827	valid_1's auc: 0.934338
[675]	training's auc: 0.978869	valid_1's auc: 0.934331
[676]	training's auc: 0.978897	valid_1's auc: 0.934331
[677]	training's auc: 0.978923	valid_1's auc: 0.934331
[678]	training's auc: 0.978968	valid_1's auc: 0.934323
[679]	training's auc: 0.979006	valid_1's auc: 0.934328
[680]	training's auc: 0.979036	valid_1's auc: 0.93436
[681]	training's auc: 0.979076	valid_1's auc: 0.934363
[682]	training's auc: 0.979115	valid_1's auc: 0.934301
[683]	training's auc: 0.979134	valid_1's auc: 0.934306
[684]	traini

...

[504]	training's auc: 0.971971	valid_1's auc: 0.93873
[505]	training's auc: 0.972019	valid_1's auc: 0.938703
[506]	training's auc: 0.972067	valid_1's auc: 0.938703
[507]	training's auc: 0.972103	valid_1's auc: 0.938714
[508]	training's auc: 0.972156	valid_1's auc: 0.938773
[509]	training's auc: 0.972203	valid_1's auc: 0.938756
[510]	training's auc: 0.972236	valid_1's auc: 0.938756
[511]	training's auc: 0.972277	valid_1's auc: 0.938765
[512]	training's auc: 0.97231	valid_1's auc: 0.938752
[513]	training's auc: 0.972352	valid_1's auc: 0.93874
[514]	training's auc: 0.972407	valid_1's auc: 0.938752
[515]	training's auc: 0.972439	valid_1's auc: 0.938754
[516]	training's auc: 0.972481	valid_1's auc: 0.938752
[517]	training's auc: 0.972527	valid_1's auc: 0.938731
[518]	training's auc: 0.972565	valid_1's auc: 0.938709
[519]	training's auc: 0.972598	valid_1's auc: 0.938707
[520]	training's auc: 0.972633	valid_1's auc: 0.938709
[521]	training's auc: 0.972672	valid_1's auc: 0.938718
[522]	trainin

[666]	training's auc: 0.978305	valid_1's auc: 0.939274
[667]	training's auc: 0.978348	valid_1's auc: 0.939232
[668]	training's auc: 0.978385	valid_1's auc: 0.939187
[669]	training's auc: 0.978414	valid_1's auc: 0.939187
[670]	training's auc: 0.978444	valid_1's auc: 0.939162
[671]	training's auc: 0.978472	valid_1's auc: 0.939199
[672]	training's auc: 0.978508	valid_1's auc: 0.939209
[673]	training's auc: 0.978533	valid_1's auc: 0.939253
[674]	training's auc: 0.978564	valid_1's auc: 0.939243
[675]	training's auc: 0.978594	valid_1's auc: 0.939268
[676]	training's auc: 0.978624	valid_1's auc: 0.93927
[677]	training's auc: 0.978644	valid_1's auc: 0.93926
[678]	training's auc: 0.978672	valid_1's auc: 0.939319
[679]	training's auc: 0.978709	valid_1's auc: 0.939361
[680]	training's auc: 0.978758	valid_1's auc: 0.939347
[681]	training's auc: 0.978818	valid_1's auc: 0.939375
[682]	training's auc: 0.978857	valid_1's auc: 0.939354
[683]	training's auc: 0.978894	valid_1's auc: 0.939369
[684]	traini

[819]	training's auc: 0.982902	valid_1's auc: 0.93938
[820]	training's auc: 0.982918	valid_1's auc: 0.939389
[821]	training's auc: 0.982933	valid_1's auc: 0.939366
[822]	training's auc: 0.982965	valid_1's auc: 0.939376
[823]	training's auc: 0.982989	valid_1's auc: 0.939377
[824]	training's auc: 0.983008	valid_1's auc: 0.939377
[825]	training's auc: 0.983036	valid_1's auc: 0.939363
[826]	training's auc: 0.983066	valid_1's auc: 0.93941
[827]	training's auc: 0.983083	valid_1's auc: 0.939394
[828]	training's auc: 0.983112	valid_1's auc: 0.939344
[829]	training's auc: 0.983143	valid_1's auc: 0.939334
[830]	training's auc: 0.98317	valid_1's auc: 0.939316
[831]	training's auc: 0.983201	valid_1's auc: 0.939349
[832]	training's auc: 0.983221	valid_1's auc: 0.939375
[833]	training's auc: 0.983253	valid_1's auc: 0.939337
[834]	training's auc: 0.983269	valid_1's auc: 0.939331
[835]	training's auc: 0.983293	valid_1's auc: 0.939347
[836]	training's auc: 0.983307	valid_1's auc: 0.939334
[837]	trainin

[91]	training's auc: 0.945714	valid_1's auc: 0.921059
[92]	training's auc: 0.945826	valid_1's auc: 0.921106
[93]	training's auc: 0.945935	valid_1's auc: 0.921039
[94]	training's auc: 0.946157	valid_1's auc: 0.921339
[95]	training's auc: 0.946265	valid_1's auc: 0.921357
[96]	training's auc: 0.946421	valid_1's auc: 0.921497
[97]	training's auc: 0.946493	valid_1's auc: 0.921365
[98]	training's auc: 0.94664	valid_1's auc: 0.921455
[99]	training's auc: 0.946833	valid_1's auc: 0.921674
[100]	training's auc: 0.946914	valid_1's auc: 0.921637
[101]	training's auc: 0.946991	valid_1's auc: 0.921608
[102]	training's auc: 0.947101	valid_1's auc: 0.921615
[103]	training's auc: 0.947183	valid_1's auc: 0.921654
[104]	training's auc: 0.947341	valid_1's auc: 0.921703
[105]	training's auc: 0.947455	valid_1's auc: 0.921672
[106]	training's auc: 0.947591	valid_1's auc: 0.921615
[107]	training's auc: 0.947648	valid_1's auc: 0.921706
[108]	training's auc: 0.947796	valid_1's auc: 0.921891
[109]	training's auc

[258]	training's auc: 0.959546	valid_1's auc: 0.922923
[259]	training's auc: 0.959637	valid_1's auc: 0.922957
[260]	training's auc: 0.959686	valid_1's auc: 0.922875
[261]	training's auc: 0.959758	valid_1's auc: 0.922864
[262]	training's auc: 0.959837	valid_1's auc: 0.923006
[263]	training's auc: 0.95989	valid_1's auc: 0.922975
[264]	training's auc: 0.959926	valid_1's auc: 0.923007
[265]	training's auc: 0.959981	valid_1's auc: 0.922923
[266]	training's auc: 0.960032	valid_1's auc: 0.922913
[267]	training's auc: 0.960106	valid_1's auc: 0.922955
[268]	training's auc: 0.960174	valid_1's auc: 0.922918
[269]	training's auc: 0.960234	valid_1's auc: 0.922884
[270]	training's auc: 0.960283	valid_1's auc: 0.922828
[271]	training's auc: 0.960392	valid_1's auc: 0.923003
[272]	training's auc: 0.96049	valid_1's auc: 0.922965
[273]	training's auc: 0.96055	valid_1's auc: 0.922926
[274]	training's auc: 0.960599	valid_1's auc: 0.922961
[275]	training's auc: 0.960659	valid_1's auc: 0.922896
[276]	trainin